# Question 1

This script compares the approaches of separately filtering then maximally downsampling versus applying a polyphase downsampling filter.

In [ ]:
from pathlib import Path

import numpy as np
import scipy.fft as fft
import scipy.signal as signal

import matplotlib.pyplot as plt
import seaborn as sns

from a3_config import A3_ROOT, SAVEFIG_CONFIG

In [ ]:
# Define filter specifications

FS     = 40     # sampling frequency, kHz
F_PASS = 0.2    # cutoff frequency, kHz
F_STOP = 0.3    # stop band frequency, kHz
A_PASS = 3      # pass band attenuation, dB
A_STOP = 100    # stop band attenuation, dB

### Construct Signal

In [ ]:
# Create signal with tones at: 50, 150, 950, 1050 Hz sampled at 40 kHz
t_signal = np.arange(0, 50, 1 / FS)
x_signal = np.sin(2 * np.pi * 0.05 * t_signal) + \
    np.sin(2 * np.pi * 0.15 * t_signal) + \
    np.sin(2 * np.pi * 0.95 * t_signal) + \
    np.sin(2 * np.pi * 1.05 * t_signal)

f_signal = fft.fftfreq(8192, 1 / FS)[:4096]
h_signal = fft.fft(x_signal, 8192)[:4096]

fig, axs = plt.subplots(2, figsize=(6, 3))
fig.tight_layout()

sns.lineplot(x=t_signal, y=x_signal, ax=axs[0])
sns.lineplot(x=f_signal, y=np.abs(h_signal), ax=axs[1])

axs[0].set_xlabel("Time (ms)")
axs[1].set_xlabel("Frequency (kHz)")
axs[1].set_xlim([0, 2.5])

# fname = Path(A3_ROOT, "output", "q1_signal.png")
# fig.savefig(fname, **SAVEFIG_CONFIG)
plt.show()

### Apply Kaiser LPF

In [ ]:
ripple_p = 1 - np.power(10, -A_PASS / 20)
ripple_s = np.power(10, -A_STOP / 20)
print("Maximum pass band ripple:", ripple_p)
print("Maximum stop band ripple:", ripple_s)

A = -20 * np.log10(min(ripple_p, ripple_s))
print("Required attenuation:", A, "dB")

In [ ]:
# Kaiser window filter length estimate
N = int(np.ceil((A - 7.95)/(14.36 * ((F_STOP - F_PASS) / FS))))
N = N + 1 if (N % 2) else N
print("Filter length estimate:", N)

beta = 0.1102 * (A - 8.7)
print("Kaiser window beta:", beta)

Construct a vector representing the ideal frequency response.

In [ ]:
# Calculate pass band width, L
L = int(np.round(N * (F_PASS) / FS))
print("Bins in passband:", L)

# Construct V, with 1's in the pass band and 0's in the stop band
h_ideal = np.zeros(N//2)
h_ideal[:L] = np.ones(L)
h_ideal = np.concatenate([h_ideal, np.flip(h_ideal)])

# Construct a frequency axis for plotting
f_ideal = np.linspace(0, FS, N)

# Plot ideal frequency response, represented by vector V
fig, ax = plt.subplots(figsize=(6, 2))
fig.tight_layout()

sns.lineplot(x=f_ideal[:N//2], y=h_ideal[:N//2], ax=ax)
ax.set_xlim([0, 1])
ax.set_xlabel("Frequency (kHz)")
ax.set_ylabel("Gain")

plt.show()

In [ ]:
# Helper function for converting frequency response to dB scale
dB = lambda x: 20 * np.log10(x)

def plot_freqz(w, h, fname=None):
    """Plot frequency response and overlay filter requirements."""
    fig, ax = plt.subplots(figsize=(6, 3))
    fig.tight_layout()
    sns.lineplot(x=w, y=dB(np.abs(h)), ax=ax)
    # Plot pass band requirement
    ax.axhline(-3, c="g", lw=0.5, label="Pass band requirement")
    ax.axvline(0.3, c="g", lw=0.5)
    # Plot pass band requirement
    ax.axhline(-100, c="r", lw=0.5, label="Stop band requirement")
    ax.axvline(0.5, c="r", lw=0.5)
    # Axis labels
    ax.set_xlabel("Frequency (kHz)")
    ax.set_ylabel("Gain (dB)")
    ax.legend(framealpha=1)
    # Save or just show
    if fname:
        fig.savefig(Path(A3_ROOT, "output", fname), **SAVEFIG_CONFIG)
    plt.show()

In [ ]:
# Impulse (time) response of ideal filter
x_ideal = fft.fftshift(fft.ifft(h_ideal))

# Construct and apply the Kaiser window
x_windowed = x_ideal * signal.windows.kaiser(N, beta)
h_windowed = fft.fft(x_windowed, 512)[:256]

# Construct frequency axis for plotting
f_windowed = np.linspace(0, FS / 2, 256)

plot_freqz(f_windowed, h_windowed)

In [ ]:
# Apply filter to signal, removing transient edge effects
x_filt = signal.convolve(x_windowed, x_signal)[N//2:-(N//2-1)]
h_filt = fft.fft(x_filt, 8192)[:4096]

fig, axs = plt.subplots(2, figsize=(6, 3))
fig.tight_layout()

sns.lineplot(x=t_signal, y=x_filt.real, ax=axs[0])
sns.lineplot(x=f_signal, y=np.abs(h_filt), ax=axs[1])

axs[0].set_xlabel("Time (ms)")
axs[1].set_xlabel("Frequency (kHz)")
axs[1].set_xlim([0, 2.5])

# fname = Path(A3_ROOT, "output", "q1_filt.png")
# fig.savefig(fname, **SAVEFIG_CONFIG)
plt.show()

### Maximally Downsample

In [ ]:
M = int(FS // (F_PASS + F_STOP))
print("Downsampling by factor of:", M)

x_dsamp = x_filt[::M]
h_dsamp = fft.fft(x_dsamp, 8192)[:4096]

t_dsamp = np.arange(0, 50, M / FS)
f_dsamp = fft.fftfreq(8192, M / FS)[:4096] * 1000 # show in Hz rather than kHz

fig, axs = plt.subplots(2, figsize=(6, 3))
fig.tight_layout()

sns.lineplot(x=t_dsamp, y=x_dsamp.real, ax=axs[0])
sns.lineplot(x=f_dsamp, y=np.abs(h_dsamp), ax=axs[1])

axs[0].set_xlabel("Time (ms)")
axs[1].set_xlabel("Frequency (Hz)")

# fname = Path(A3_ROOT, "output", "q1_dsamp.png")
# fig.savefig(fname, **SAVEFIG_CONFIG)
plt.show()

### Polyphase Downsample

In [ ]:
# TODO